In [1]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import norm

# ============================== Core Functions ==============================
def black_scholes_call(S0: float, K: float, T: float, r: float, sigma: float) -> float:
    """Compute European call option price using Black-Scholes formula."""
    d1 = (np.log(S0/K) + (r + 0.5*sigma**2)*T) / (sigma*np.sqrt(T))
    d2 = d1 - sigma*np.sqrt(T)
    return S0 * norm.cdf(d1) - K * np.exp(-r*T) * norm.cdf(d2)

def log_space_euler_call_price_vec(M: int, N: int, S0: float, K: float, T: float, r: float, sigma: float) -> float:
    """Vectorized log-space Monte Carlo simulation for European call option."""
    dt = T / N
    Z = np.random.normal(0, 1, size=(M, N))
    X = np.log(S0) + ((r - 0.5*sigma**2)*dt + sigma*np.sqrt(dt)*Z).sum(axis=1)
    S_T = np.exp(X)
    payoff = np.maximum(S_T - K, 0)
    return np.exp(-r*T) * np.mean(payoff)

def direct_euler_call_price(M: int, N: int, S0: float, K: float, T: float, r: float, sigma: float) -> float:
    """Euler-Maruyama discretization for original SDE."""
    dt = T / N
    S = np.full(M, S0)
    for _ in range(N):
        Z = np.random.normal(0, 1, M)
        S += r*S*dt + sigma*S*np.sqrt(dt)*Z
    payoff = np.maximum(S - K, 0)
    return np.exp(-r*T) * np.mean(payoff)

# ============================== Analysis Functions ==============================
def plot_convergence(bs_price: float, params: dict):
    """Generate convergence analysis plots with error visualization."""
    # Unpack parameters
    M_values = params["M_values"]
    N_log_space = params["N_log_space"]
    M_fixed = params["M_fixed"]
    N_values = params["N_values"]
    S0, K, T, r, sigma = params["S0"], params["K"], params["T"], params["r"], params["sigma"]

    # Plot 1: Log-space convergence vs paths
    plt.figure(figsize=(12, 5))
    plt.subplot(1, 2, 1)
    for N in N_log_space:
        errors = []
        for M in M_values:
            price = log_space_euler_call_price_vec(M, N, S0, K, T, r, sigma)
            errors.append(price - bs_price)
        plt.plot(M_values, errors, marker='o', markersize=4, label=f'N={N}', alpha=0.7)
    plt.axhline(0, color='black', linestyle='--')
    plt.xscale('log')
    plt.xlabel('Number of Paths (M)')
    plt.ylabel('Price Error (MC - BS)')
    plt.title('Log-Space: Error vs Paths')
    plt.legend()
    plt.grid(True, linestyle='--', alpha=0.7)

    # Plot 2: Original SDE convergence vs time steps
    plt.subplot(1, 2, 2)
    errors = []
    for N in N_values:
        price = direct_euler_call_price(M_fixed, N, S0, K, T, r, sigma)
        errors.append(price - bs_price)
    plt.semilogx(N_values, errors, 'bo-', markersize=6)
    plt.axhline(0, color='black', linestyle='--')
    plt.xlabel('Number of Time Steps (N)')
    plt.ylabel('Price Error (MC - BS)')
    plt.title(f'Original SDE: Error vs Time Steps (M={M_fixed})')
    plt.grid(True, linestyle='--', alpha=0.7)

    plt.tight_layout()
    plt.show()



In [ ]:
# ============================== Main Function ==============================
def main():
    """Main execution routine."""
    # Parameter settings
    params = {
        "S0": 100.0,
        "K": 100.0,
        "T": 1.0,
        "r": 0.05,
        "sigma": 0.2,
        "M_values": np.unique(np.logspace(2, 5, num=50).astype(int)),
        "N_log_space": [1, 10, 100],
        "M_fixed": 100_000,
        "N_values": [1, 2, 5, 10, 20, 50, 100, 200, 500, 1000]
    }
    np.random.seed(42)

    # Compute reference price
    bs_price = black_scholes_call(params["S0"], params["K"], params["T"],
                                 params["r"], params["sigma"])
    print(f"[Exact Price] Black-Scholes: {bs_price:.4f}")

    # Run convergence analysis
    plot_convergence(bs_price, params)

    # Demonstrate high-accuracy results with CORRECT parameter passing
    print("\n[High-Accuracy Simulations]")
    print(f"Log-Space (N=1, M=1e6): {log_space_euler_call_price_vec(1_000_000, 1, params['S0'], params['K'], params['T'], params['r'], params['sigma']):.4f}")
    print(f"Original SDE (N=500, M=1e5): {direct_euler_call_price(100_000, 500, params['S0'], params['K'], params['T'], params['r'], params['sigma']):.4f}")

if __name__ == "__main__":
    main()

[Exact Price] Black-Scholes: 10.4506


### 📌 Comment: Monte Carlo Convergence for European Call Pricing

This experiment evaluates the convergence of Monte Carlo estimators for a European call option priced under the Black-Scholes model.

**Settings:**
- $S_0 = 100.0$, $K = 100.0$, $T = 1.0$
- $r = 0.05$, $\sigma = 0.2$

---

**Reference Price (Black-Scholes Closed Form):**  
$C_{BS}$ = 10.4506

---

**Log-Space Simulation (Exact in 1 Step):**  
With $N = 1$ and $10^6$ paths:  
$C_{MC}$ = 10.4342

**Original SDE Simulation:**  
With $N = 500$ and $10^5$ paths:  
$C_{MC}$ = 10.3941

---

**Conclusion:**
- **Log-space simulation with $N = 1$** provides the most efficient and exact discretization because the log-normal terminal distribution is sampled directly.
- **Original SDE simulation** requires significantly more time steps to converge and introduces discretization error.
- Euler discretization in log-space is **more robust and faster**, especially for vanilla European options with known exact dynamics.
